---
# GA DSI6 capstone project - Predicting song genres by their lyrics
### Oliver Sildmäe
<br>
## Executive Summary

This is a high-level summary detailing the data used and various results achieved. For the full documentation and results of this project, along with the code, please see the other notebook in this repository.

---

## Problem statement and goal

Genres are the primary way in which we categorise music. They identify music as belonging to a certain shared tradition or set of conventions, providing people with a rough idea of what to expect from a song even before listening to it. This labeling is usually done by identifying how music 'sounds' - the instruments used, song running times, keys and tempos, etc.

Lyrics are generally very much secondary in this categorisation. Even though people have certain stereotypes about music of certain genres (e.g., Christian rock being largely about Jesus; punk about anarchy, or hip-hop using a lot of profanities and putting large emphasis on complex rhyming schemes), it seems uncertain that just by looking at the lyrics of a song, one would be able to tell whether the song belongs to, for example, the rock, pop or country genre, or something different altogether. Perhaps there are, however, common lyrical themes within these genres, ones that would be difficult to discern just by looking at individual songs?

This project attempts to answer that question by using a large dataset of song lyrics and their respective genre labels, examining the possibility of predicting songs' genres based purely on their lyrical content. If successful, it would give insight into whether songs of a specific genre tend to have lyrics that significantly differentiate them for songs of other genres. Further, it would provide us with a comparative element of genre relatedness based on lyrics, as well as helping us identify which words (or the lack thereof) help define certain genres.

This will be a multiple classification problem, with each genre/style as a separate class within the dependent variable.

The data for this project comes from the below two auxiliary projects to The Million Song Dataset:<br>
Genres - http://www.ifs.tuwien.ac.at/mir/msd/download.html <br>
Lyrics - https://labrosa.ee.columbia.edu/millionsong/musixmatch

## Data set-up and cleaning

The lyrics data comes in a SQLite database (source above). It identifies each track by their MSD (Million Song Dataset) ID and provides the bag-of-words format word counts for each of these tracks. Note that not all lyrics are available but rather the overall top 5000 words across all songs in the dataset are provided. The words have been stemmed.

The genres data comes in a text file, simply having one of these same MSD track IDs and the relevant genre label (from All Music Guide; source above) on each row. 

The main goal here is to merge the two datasets, using their shared song ID-s to match genres and lyrics, and to set the data up for exploratory data analyses and modeling.

## Exploratory data analysis

After merging the genres and lyrics data, as well as splitting the resulting dataframe according to a a 1:3 test to train ratio, the training dataset is left with 78336 songs (rows), with the top 5000 most frequent words across all songs for columns. The columns are here read in as binary, simply indicating whether a given word is present in the song's lyrics or not.<br><br>
The dependent variable has 25 possible categories (genres), the distribution of which can be seen in the graph below:

![title](Assets/IMG1.png)

The style labels in the dataset seem to be rather unevenly split - for example, there are far more songs that have various metal, rock or pop labels than there are classical, jazz or big band songs. 
<br>
<br>
If not countered with over- and/or undersampling, the models built will be biased towards these oft-occurring genres, rarely predicting the styles which appear less frequently. It's an interesting question whether the above distribution reflects reasonably well what we expect an average sample of random songs to be, and therefore whether this imbalance is a serious issue. 
<br>
<br>
The underlying track selection for the Million Song Dataset comes from The Echo Nest, a music intelligence and data platform founded in 2005. The genre/style data comes from All Music Guide, an online music guide launched in 1991. As both of these website primarily focus on contemporary music, the dataset is very likely biased towards music from the past 20-30 years. This isn't necessarily an issue though, as we expect the model to be used primarily in the context of contemporary music.
<br>
<br>
Further, certain genres like hip-hop/rap and R&B seem to be very under-represented. Despite these two genres combined accounting for about a quarter of US music consumption in 2017 (https://www.nielsen.com/us/en/insights/reports/2018/2017-music-us-year-end-report.html), they account for less than 7% of the dataset at hand. Thus, we assume the model to be somewhat disproportionately undertuned to predicting for example hip-hop/rap/R&B music, instead making more predictions for rock, metal and pop songs.
<br>
<br>
Finally, there is also the more general issue that genres are inherently rather arbitrary, with a lot of cross-over between various genres. Therefore, for many songs, the 'real' genre they belong to can be very hard to pin down, and we are here relying purely on the labels attributed at by All Music Guide. Also, although we might be partially right in these borderline cases, the models will be making a single (albeit probabilistic) classification.

#### Lyrical profile comparisons of death metal and gospel music

In the word clouds below are displayed the most common words within the respective genres. The size of the word in the cloud indicates its relative importance.

![title](Assets/IMG2.png)

![title](Assets/IMG3.png)

Once stop-words are removed, the expectation of what the lyrics for death metal and gospel music ought to be holds largely true. Death metal lyrics focus largely on death and mortality, with words like 'blood', 'soul', 'pain' and 'death' standing out as oft-occurring themes. Conversely, for gospel music, words such as 'love', 'lord', 'heart' and 'god' are among the most often encountered ones. There are also common shared themes like 'life' and 'time'.<br><br>
This is reassuring as it will be these lyrical stereotypes which will help our models attribute the correct genres to songs.

## Data transformations and related modeling

### Models with all (unbalanced) genre labels

Below, a number of different classifiers will be tested. As many models' performances are highly dependent on using an appropriate set of hyperparameters, these parameters will for all models here be gridsearched and cross-validated using GridSearchCV.

![title](Assets/IMG4.png)

<br>
Based on the cross-validated training accuracy scores, logistic regression seems to be the winner, and will be the primary model used going forward. It is, however, possible that the MLP or XGBoost classifiers' performances could be still improved on by more extensive hyperparameter optimisation.

Although the best cross-validated training score of about 0.283 means that the model will in this case make a right prediction just above a quarter of the times, it is important to bear in mind that there are 25 labels (genres) in this classification problem. With the baseline being just over 0.08, the score of 0.283 is therefore nevertheless a significant improvement in prediction accuracy. 

Importantly, looking at the genre labels, it is immediately clear that certain genres are more similar to one another than others. For instance, rock  music is broken down into five sub-genres and metal into three sub-genres, whereas rap/hip-hop is simply a single label. It is arguably more 'forgivable' to make mis-classifications of similar sub-genres than it is to make mis-classifications for a genre that is completely dissimilar. For example, mistaking alternative rock music for contemporary rock music is far less severe than mistaking it for electronica or grunge music.

It is therefore important to look at the confusion matrix, as well as the precision and recall scores, to see where the misclassifications are being made.
<br><br>

![title](Assets/IMG0.png)

In the colour-coded confusion matrix above, the colours schemes are per each column (genre), indicating for which class the predictions were made. One would hope to see a diagonal downwards red line surrounded by green, indicating that the correct predictions have been made for each class, with little or no misclassifications.<br>

The results are reassuring as the red/orange diagonal is clearly visible. What stands out is the orange zone in the bottom-right corner, indicating that various rock genres do indeed seem to get mixed up a lot with one another. We could consider addressing that by resampling and merging/consolidating these potentially very similar labels.

### Models with under/oversampling

As discussed above in the EDA section, it is unlikely that the data genre label distribution reflects that of the true world. It could therefore be informative to create models less biased towards the most popular genres in the database by resampling the data.<br><br>
Although this will almost certainly bring down the overall model accuracy for the test data, it would help create a models which are more sensitive to the genres that are less represented in the database.

##### Undersampling

As expected with undersampling, the overall accuracy falls significantly, as I've thrown out a lot of data in order to undersample all the classes. However, we generally see recall rise and precision fall for classes that were previously under-represented. This is to be expected, as we are now making more predictions for these classes, many of which will be false positives, leading to a reduction in precision. Conversely, recall rises for these classes because we are getting fewer false negatives by actually predicting that class.

##### Under - and oversampling ('middlesampling') all classes to 3070 samples

It is important to note that when undersampling above, we are throwing out the vast majority of our overall data due to a couple of genres (e.g. big band) having very few songs. It could perhaps make more sense to meet at half-way, undersampling some of the more popular genres and oversampling some of the less popular ones. This is what's attempted below

The best model here (logistic regression, with accuracy of about 0.262) performs almost as well as the logistic regression above for the original unbalanced dataset. Considering that the baseline is now only 0.04 due to there being the same amount of samples in all classes, the model arguably performs relatively much better than the one above! 

Whether this model would be preferred to the original one in case of an expectation of more balanced classes depends on how precision and recall have changed for the under-represented classes, compared to the original model. This will be investigated below:

![title](Assets/IMG5.png)

![title](Assets/IMG6.png)

For the above two graphs, red bars represent classes which were undersampled down to 3070 and blue bars represent classes which were oversampled to that same sample size. Note that the amount of samples 'gained' or 'lost' with the over - and undersampling varies greatly within each colour, as some labels started off with more observations and other less.

Apart from the obvious outlier of International folk music, which needs to be investigated further, the expectation of falling precision and rising recall for the classes that were oversampled holds true, as does the opposite for the undersampled classes. This means that we have successfully built a model that, with similar accuracy, is more sensitive to the classes that were previously under-represented in the dataset. 

Whether this model is an improvement on the one built on the original imbalanced dataset depends on one's belief in whether the equal distribution of song labels in the latter model is more in line with our expectaion of genre labels 'in the real world'.

### Models with TFIDF

TFIDF, short for 'term frequency–inverse document frequency', is a numerical statistic used to reflect how important words are to a document in a corpus ("Data Mining", Rajaraman, A. and Ullman, J.D. (2011)).

This could potentially perform better than the previously used binary lyric variables, as TFIDF gives weights to words that are specific to certain labels, thereby helping to highlight the importance of words that set some classes/genres apart from others.

The one potential issue here is that for TFIDF we would ideally like to have all lyrics across all songs, not just the top 5000.

Even without hyperparameter optimisation, TFIDF logistic regression improved both the cross-validated training scores and the test score when compared to the initial logistic regression model above.

### Models with consolidated style labels

Because of the aforementioned problem that some of the genre labels are presumably rather similar to one another, both in how people perceive them and also where their respective misclassifications lie, it could make sense to merge some of these subgenres into larger ones. 

The most obvious examples here are five rock subgenres, which most people probably wouldn't be able to distinguish from one another, and where most of the misclassifications are made for other rock subgenres (see confusion matrix above)

![title](Assets/IMG7.png)

##### Models with imbalanced data (consolidated genres)

These models are, as expected, performing much better than the ones above, with logistic regression training and test scores of around 0.45. Of course, the baseline is now considerably higher as well, at around 0.31. 

Therefore, if distinguishing the various metal and rock subgenres is not beneficial to the user of the model, then these models might be preferable to the original ones above. However, obviously comes at the expense of being able to tell apart various rock and metal subgenre songs from one another.

##### Models with data rebalanced by undersampling (consolidated genres)

The problem of class imbalance is obviously now even more stark. To counter that, the classes could, like above, be resampled in order to make the minority classes more pertinent. The results for the performance of logistic regression for such data can be seen below.

Copared to the model above, accuracy has fallen about 10%, despite the baseline being by about 20% lower. Again, if one wished to build a model more sensitive to the minority genres, this model would likely be preferable. 

![title](Assets/IMG8.png)

### Hyperparameter optimisation for the final model (consolidated labels + TFIDF)

Choosing the most appropriate dataset for the final model from the above transformations will be largely subjective. Although the accuracy increase in the case of consolidated rock and metal subgenres could be justifiable in cases where we don't deem this distinction to be important, there are certainly also other cases where these distinctions would be informative or even vital.

Here, I will be choosing the TFIDF data with consolidated metal and rock labels, as models on this data achieved the best results, and I am not interested in telling apart rock and metal genres to such a granular level.

The models that initially after hyperparameter optimisation performed best were logistic regression, XGBoost classifier and MLP. MLP has currently not been properly gridsearched here simply due to computational costs. However, using the optimal hyperparameters from above did not yield better results than logistic regression or XGBoost. 

The hyperparameters for these remaining two algorithms will be fine-tuned below.

![title](Assets/IMG9.png)

The accuracy scores for logistic regression and XGBoost are remarkably similar here, with logistic regression performing maginally better on the (crossvalidated) training dataset and XGBoost on the test set. As logistic regression nevertheless outperformed XGBoost here on the cross-validated training dataset with a score of about 0.447, and it works slightly faster at producing results, it will be used as the final model for genre predictions at:<br>
http://ec2-52-14-243-207.us-east-2.compute.amazonaws.com/

### Dimensionality reduction and clustering

##### PCA

To employ various clustering techniques on data of this size, it would be helpful or even necessary to use principal component analysis (PCA) to reduce the amount of variables we're dealing with

Looks like dimensionality reduction might not be very useful given that even 50 variables explain less than 10% of the variance in the data. A number of clustering algorithms were tried (including DBScan and Hierarchical clustering), yet due to the amount of features present without successful PCA, these tasks turned out to be too resource intensive to calculate.

Instead, Latent Dirichlet Allocation (https://en.wikipedia.org/wiki/Latent_Dirichlet_allocation) will be used below:

##### Latent Dirichlet Allocation

Latent Dirichlet allocation has proven to be rather good at identifying common themes among songs. Looking at the first group of songs, it has labelled together the 'international' songs belonging primarily to the Latin pop and international folk genres. Looking at topic 2, it has succeeded in catching most of the hip-hop/rap songs. The abovementioned common themes among various metal genres have seemingly been the primary driver behind the allocation of topic 3.  

These allocations and/or clustering could be used in the future to create novel genre groupings of songs based purely on their lyrical content.

## Conclusions: findings and limitations

The purpose of this project was to examine whether songs' lyrics are a good predictor of their genres. Given the dataset, it was going to be a difficult task. The main difficulty arises from the fact that the original dataset has 25 genre labels. Some of those genres tend to be very distinct from one another, however, other are also very similar to one another in their lyrical content, being subgenres of a larger one (e.g. the five very similar sub-genres of rock music). Thus, this lack of clear-cut classes, combined with the large number of possible classification labels, makes it inherently very difficult to build a highly accurate model.

However, I was nevertheless able to build models that performed reasonably well, more than tripling the original accuracy of the baseline of around 0.083. Looking at the logistic regression model (not resampled), the model built is rather good at predicting latin pop songs and hip-hop/rap songs, with precision and recall scores of over 0.6 for both of them. The former is very clearly easy to distinguish due to it being the only genre with predominantly non-English lyrics. Hip-hop songs are likely distinguishable due to their famously unique and comprehensive vocabularies, as well as the use of profanities highlighted by the most pertinent words in the relevant LDA group above.

However, due to the relatively low precision scores for other classes, predictions for them should be taken with a pinch of salt. The misclassifications aren't however, for the most part, as bad as they seem, with a very large proportion of them being made for similar classes. Most notably, across all five rock sub-genres, most misclassifications are made for other rock classes. As there is, as previously mentioned, a lot of overlap and a lack of clear distinction in case of these related sub-genres, many of these misclassifications might be acceptable to users of the model, being 'close enough'.

These models could likely hbe greatly improved by introducing some additional variables about the songs available in the Million Song Dataset, like the key the song is in, its loudness, tempo or length. It could also have been very useful to have access to the original lyrics data, to see if using n-grams rather than simple binary single word counters could have improved the performance of the models.

If one were interested in predicting whether a song belongs to certain genres or not, the dataset at hand could easily be altered to simply make a binary prediction about whether a song belongs to a certain genre (e.g. hip-hop vs. rock) or not.

Finally, the real-life applicability of the model could likely be improved by using a dataset which better reflects the expected genre distribution of contemporary music. As mentioned above, such a dataset would, for example, have a much higher proportion of R&B and hip-hop/rap songs that the dataset at hand.